In [1]:
import pandas as pd
import numpy as np
import glob

import os
import tensorflow as tf   
from sklearn.preprocessing import scale
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses

In [2]:
# transpose for tied autoencoder
class DenseTranspose(tf.keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        self.dense = dense
        self.activation = tf.keras.activations.get(activation)
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name='bias',shape=[self.dense.input_shape[-1]],initializer="zeros")
        super().build(batch_input_shape)
    def call(self, inputs):
        z = tf.matmul(inputs, self.dense.weights[0], transpose_b = True)
        return self.activation(z + self.biases)

In [3]:
# gps model
input_o = tf.keras.layers.Input(shape=(4), name='input_layer_gps')
dense1 = layers.Dense(4, activation='relu')
dense2 = layers.Dense(4, activation='relu')
dense3 = layers.Dense(4, activation='relu')
dense4 = layers.Dense(4, activation='relu')
models = dense1(input_o )
models = dense2(models)
models = dense3(models)
models = dense4(models)
model = tf.keras.Model(inputs=input_o, outputs=models)

input_oo = tf.keras.layers.Input(shape=(4), name='input_layer0_gps')
models2 = DenseTranspose(dense4, activation = 'relu')(input_oo)
models2 = DenseTranspose(dense3, activation = 'relu')(models2)
models2 = DenseTranspose(dense2, activation = 'relu')(models2)
models2 = DenseTranspose(dense1, activation = 'relu')(models2)
model2 = tf.keras.Model(inputs=input_oo, outputs=models2)

input = tf.keras.layers.Input(shape=(10,4), name='input_layer1_gps')
input1 = layers.Lambda(lambda x: x[:,0,:], output_shape=(1))(input)
input2 = layers.Lambda(lambda x: x[:,1,:], output_shape=(1))(input)
input3 = layers.Lambda(lambda x: x[:,2,:], output_shape=(1))(input)
input4 = layers.Lambda(lambda x: x[:,3,:], output_shape=(1))(input)
input5 = layers.Lambda(lambda x: x[:,4,:], output_shape=(1))(input)
input6 = layers.Lambda(lambda x: x[:,5,:], output_shape=(1))(input)
input7 = layers.Lambda(lambda x: x[:,6,:], output_shape=(1))(input)
input8 = layers.Lambda(lambda x: x[:,7,:], output_shape=(1))(input)
input9 = layers.Lambda(lambda x: x[:,8,:], output_shape=(1))(input)
input10= layers.Lambda(lambda x: x[:,9,:], output_shape=(1))(input)
model_1 = model(input1)
model_2 = model(input2)
model_3 = model(input3)
model_4 = model(input4)
model_5 = model(input5)
model_6 = model(input6)
model_7 = model(input7)
model_8 = model(input8)
model_9 = model(input9)
model_10= model(input10)
merge_layer = tf.keras.layers.concatenate(inputs=[model_1, model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_9,model_10])
dense5 = layers.Dense(32, activation='relu')
dense6 = layers.Dense(24, activation='relu')
dense7 = layers.Dense(16, activation='relu')
model_encoder = dense5(merge_layer)
model_encoder = dense6(model_encoder)
model_encoder = dense7(model_encoder)
model_down_gps = tf.keras.Model(inputs=[input], outputs=model_encoder,name = "encoder")

# wifi model
input = tf.keras.layers.Input(shape=(15), name='input_layer1_wifi')
dense1 = tf.keras.layers.Dense(32, activation='relu')
dense2 = tf.keras.layers.Dense(24, activation='relu')
dense3 = tf.keras.layers.Dense(16, activation='relu')
model_encoder = dense1(input)
model_encoder = dense2(model_encoder)
model_encoder = dense3(model_encoder)
model_down_wifi = tf.keras.Model(inputs=[input], outputs=model_encoder,name = "encoder")

# magnetic model
input = tf.keras.layers.Input(shape=(3), name='input_layer1_mag')
dense1 = tf.keras.layers.Dense(8, activation='relu')
dense2 = tf.keras.layers.Dense(12, activation='relu')
dense3 = tf.keras.layers.Dense(16, activation='relu')
model_encoder = dense1(input)
model_encoder = dense2(model_encoder)
model_encoder = dense3(model_encoder)
model_down_mag = tf.keras.Model(inputs=[input], outputs=model_encoder,name = "encoder")

In [4]:
# load gps trained model
path = "gps_checkpoints/checkpoints_1222/"
latest = tf.train.latest_checkpoint(path)
latest
checkpoint = tf.train.Checkpoint(model_down=model_down_gps)
checkpoint.restore(latest).expect_partial()

In [5]:
# load wifi trained model
path = "wifi_checkpoints/checkpoints_1222/"
latest = tf.train.latest_checkpoint(path)
latest
checkpoint = tf.train.Checkpoint(model_down=model_down_wifi)
checkpoint.restore(latest).expect_partial()

In [6]:
# load magnetic trained model
path = "magnetic_checkpoints/checkpoints_1222/"
latest = tf.train.latest_checkpoint(path)
latest
checkpoint = tf.train.Checkpoint(model_down=model_down_mag)
checkpoint.restore(latest).expect_partial()

In [7]:
# gps data preprocess

a = 6378137
e = 8.1819190842622e-2
asq = np.power(a,2)
esq = np.power(e,2)
def ecef2lla(ecef):
    # change data from ecef coordiate to lla cordinate
    x = ecef[0]
    y = ecef[1]
    z = ecef[2]
    r = ecef[3]
    b = np.sqrt( asq * (1-esq) )
    bsq = np.power(b,2)
    ep = np.sqrt( (asq - bsq)/bsq)
    p = np.sqrt(np.power(x,2) + np.power(y,2) )
    th = np.arctan2(a*z, b*p)
    lon = np.arctan2(y,x)
    lat = np.arctan2( (z + np.power(ep,2)*b*np.power(np.sin(th),3) ), (p - esq*a*np.power(np.cos(th),3)) )
    N = a/( np.sqrt(1-esq*np.power(np.sin(lat),2)) )
    alt = p / np.cos(lat) - N
    lon = lon % (2*np.pi)
    ret = [lat, lon, alt , r]
    return ret

datas = glob.glob('transfer/*/*/*.txt')
train_data = []
train_label = [] 
for data in datas:
    if data.find('timestamp') == -1:
        f = np.loadtxt(data,delimiter=" ").copy()
        if f.size == 4:
            f = f.reshape((1,-1))
        o = np.argsort(f,axis=(0))[:,3]
        f = np.array([ecef2lla(d) for d in f])
        f = f/np.array([(2*np.pi),(2*np.pi),1e+8,1e+14])
        f = f[o]
        f.resize((10,4))
        train_data.append(f) #/100000000
        if data.split('/')[1] == "indoor":
            train_label.append("indoor"+data.split('/')[-2])
        else:
            train_label.append("outdoor"+data.split('/')[-2])
        

train_data_gps = np.array(train_data).astype('float32')
train_label_gps = np.array(pd.get_dummies(train_label)).astype('float32')

"""train_val_split = np.random.rand(len(train_data)) < 0.70
train_x = train_data[train_val_split]
train_y = train_label[train_val_split]
val_x = train_data[~train_val_split]
val_y = train_label[~train_val_split]"""

'train_val_split = np.random.rand(len(train_data)) < 0.70\ntrain_x = train_data[train_val_split]\ntrain_y = train_label[train_val_split]\nval_x = train_data[~train_val_split]\nval_y = train_label[~train_val_split]'

In [11]:
# wifi data preprocess

datas = glob.glob('20201203/wifi/dataRssi_at_*.txt')
train_data = []
origin_data = []
train_label = []
for data in datas:
    load_data = np.loadtxt(data)#f.read()
    load_data = load_data[:200,:15].astype(int)
    load_data[load_data==0] = -100
    load_data = load_data/100 + 1
    for i in range(len(load_data)):
        origin_data.append(load_data[i])
        mask = np.random.rand((15))
        mask = (mask>0.1).astype(int)
        noise = np.random.normal(scale = 0.05,size=(15))
        load_data[i]*mask
        load_data[i]+noise
        train_data.append(load_data[i])
        train_label.append(data.split("/")[-1].split("_")[-1].split(".")[0])

origin_data_wifi = np.array(origin_data).astype('float32') 
train_data_wifi = np.array(train_data).astype('float32')
train_label_wifi = np.array(pd.get_dummies(train_label)).astype('float32')

"""train_val_split = np.random.rand(len(train_data)) < 0.70
train_x = train_data[train_val_split]
train_o = origin_data[train_val_split]
train_y = train_label[train_val_split]
val_x = train_data[~train_val_split]
val_o = origin_data[~train_val_split]
val_y = train_label[~train_val_split]
BUFFER_SIZE = train_x.shape[0]
BATCH_SIZE = train_x.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_o,train_y))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valid_dataset = tf.data.Dataset.from_tensor_slices((val_x,val_o)).batch(len(val_x))"""


'train_val_split = np.random.rand(len(train_data)) < 0.70\ntrain_x = train_data[train_val_split]\ntrain_o = origin_data[train_val_split]\ntrain_y = train_label[train_val_split]\nval_x = train_data[~train_val_split]\nval_o = origin_data[~train_val_split]\nval_y = train_label[~train_val_split]\nBUFFER_SIZE = train_x.shape[0]\nBATCH_SIZE = train_x.shape[0]\ntrain_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_o,train_y))\ntrain_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)\nvalid_dataset = tf.data.Dataset.from_tensor_slices((val_x,val_o)).batch(len(val_x))'

In [12]:
# magnetic data preprocess

datas = glob.glob('20201203/wifi/dataRssi_at_*.txt')

def getRotationMatrix(gra,m):
    Ax = gra[0]
    Ay = gra[1]
    Az = gra[2]
    Ex = m[0]
    Ey = m[1]
    Ez = m[2]
    Hx = Ey*Az - Ez*Ay
    Hy = Ez*Ax - Ex*Az
    Hz = Ex*Ay - Ey*Ax
    normH = np.sqrt(Hx*Hx + Hy*Hy + Hz*Hz)
    if normH < 0.1:
        return False
    invH = 1.0 / normH
    Hx *= invH
    Hy *= invH
    Hz *= invH
    invA = 1.0 / np.sqrt(Ax*Ax + Ay*Ay + Az*Az)
    Ax *= invA
    Ay *= invA
    Az *= invA
    Mx = Ay*Hz - Az*Hy
    My = Az*Hx - Ax*Hz
    Mz = Ax*Hy - Ay*Hx
    R = np.array([Hx,Hy,Hz,Mx,My,Mz,Ax,Ay,Az])
    return R

train_data = []
train_label = []
for data in datas:
    load_data = np.loadtxt(data)#f.read()
    gras = load_data[:,-3:]/100
    ms = load_data[:,-15:-12]/100
    for gra,m in zip(gras,ms):
        R = getRotationMatrix(gra,m)
        gm = np.dot(R.reshape(3,3),m.reshape(3,1))
        train_data.append(gm)
        train_label.append(data.split("/")[-1].split("_")[-1].split(".")[0])


train_data = np.array(train_data).astype('float32')
train_data = train_data.reshape(-1,3)
train_data_mag = (train_data - train_data.mean(axis=0)) / train_data.std(axis = 0)
train_label_mag = np.array(pd.get_dummies(train_label)).astype('float32')

"""train_val_split = np.random.rand(len(train_data)) < 0.70
train_x = train_data[train_val_split]
train_y = train_label[train_val_split]
val_x = train_data[~train_val_split]
val_y = train_label[~train_val_split]
BUFFER_SIZE = train_x.shape[0]
BATCH_SIZE = train_x.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valid_dataset = tf.data.Dataset.from_tensor_slices((val_x,val_x)).batch(len(val_x))"""

'train_val_split = np.random.rand(len(train_data)) < 0.70\ntrain_x = train_data[train_val_split]\ntrain_y = train_label[train_val_split]\nval_x = train_data[~train_val_split]\nval_y = train_label[~train_val_split]\nBUFFER_SIZE = train_x.shape[0]\nBATCH_SIZE = train_x.shape[0]\ntrain_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y))\ntrain_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)\nvalid_dataset = tf.data.Dataset.from_tensor_slices((val_x,val_x)).batch(len(val_x))'